In [92]:
# Import the libraries
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style

import tensorflow as tf
from tensorflow import keras
from keras import metrics
tf.get_logger().setLevel('INFO')

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report

In [93]:
#load data
X_train = pd.read_csv('../data/processed/X_train.csv')
Y_train = pd.read_csv('../data/processed/Y_train.csv')
X_val = pd.read_csv('../data/processed/X_val.csv')
Y_val = pd.read_csv('../data/processed/Y_val.csv')
X_test = pd.read_csv('../data/processed/X_test.csv')
Y_test = pd.read_csv('../data/processed/Y_test.csv')

### Implementing Baseline

In [94]:
#show class distribution
class_counts = Y_train.value_counts().reset_index()
class_counts

,music_category,count
0,0,11662
1,2,5106
2,6,2646
3,5,2026
4,4,1787
5,7,1687
6,1,1292
7,3,1010
8,9,533
9,10,356


Since class distribution isn't too unbalanced, implement majority baseline and stratified baseline. ## EDIT class majority big now

In [95]:
#majority baseline implementation
dummy_clf = DummyClassifier(strategy = 'stratified', random_state = 42)
dummy_clf.fit(X_train, Y_train)
Y_pred = dummy_clf.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred)

print(f'Baseline Classifier Accuracy: {accuracy}')
print(f'Classification Report: {report}')

Baseline Classifier Accuracy: 0.22592905405405406
Classification Report:               precision    recall  f1-score   support

           0       0.41      0.42      0.41      3894
           1       0.04      0.03      0.03       462
           2       0.18      0.17      0.17      1695
           3       0.03      0.04      0.04       321
           4       0.06      0.07      0.06       538
           5       0.07      0.07      0.07       651
           6       0.10      0.10      0.10       917
           7       0.05      0.05      0.05       556
           8       0.02      0.02      0.02       117
           9       0.02      0.02      0.02       203
          10       0.00      0.00      0.00       118

    accuracy                           0.23      9472
   macro avg       0.09      0.09      0.09      9472
weighted avg       0.23      0.23      0.23      9472



#### Testing for uniform distribution of classes

In [96]:
min_count = class_counts['count'].min()
result = {}
unique_classes = np.unique(Y_train)
for value in unique_classes:
    indices = np.where(Y_train==value)[0][:min_count]
    result[value] = indices.tolist()
index_values = list(result.values())
index_values = [element for nestedlist in index_values for element in nestedlist]
X_train_uniform = X_train.iloc[index_values]
Y_train_uniform = Y_train.iloc[index_values]

In [97]:
X_train_uniform.info(20)

<class 'pandas.core.frame.DataFrame'>
Index: 3432 entries, 3 to 24809
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      3432 non-null   float64
 1   energy            3432 non-null   float64
 2   loudness          3432 non-null   float64
 3   mode              3432 non-null   float64
 4   speechiness       3432 non-null   float64
 5   acousticness      3432 non-null   float64
 6   instrumentalness  3432 non-null   float64
 7   liveness          3432 non-null   float64
 8   valence           3432 non-null   float64
 9   tempo             3432 non-null   float64
 10  C                 3432 non-null   float64
 11  C#                3432 non-null   float64
 12  D                 3432 non-null   float64
 13  D#                3432 non-null   float64
 14  E                 3432 non-null   float64
 15  F                 3432 non-null   float64
 16  F#                3432 non-null   float64
 17 

### Building a Model

In [98]:
len(Y_train['music_category'].unique())

11

In [99]:
X_train.shape

(28417, 22)

In [100]:
def build_model(num_features, n_classes,learning_rate=0.01):
    """Build a TF logistic regression model using Keras.
    
    Args:
    learning_rate: The desired learning rate for SGD.
    
    Returns:
    model: A tf.keras model (graph).
    """
    # Random Seed + Clear Session
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)
    
    # Build a model using keras.Sequential.
    model = keras.Sequential(name = 'Genres')
    
    # Keras layers processing
    model.add(tf.keras.layers.InputLayer(input_shape=num_features))
    
    # This layer constructs the linear set of parameters for each input feature
    # (as well as a bias), and applies a sigmoid to the result. The result is
    # binary logistic regression.
    model.add(keras.layers.Dense(
      units=n_classes,
      activation='softmax'))

    # Use the SGD optimizer as usual.
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    
    # We specify the binary_crossentropy loss (equivalent to log loss).
    # Notice that we are including 'binary accuracy' as one of the metrics that we
    # ask Tensorflow to report when evaluating the model.
    model.compile(loss='sparse_categorical_crossentropy', 
                optimizer=optimizer, 
                metrics=['accuracy'])
    
    return model

In [101]:
model = build_model(num_features = X_train.shape[1], n_classes = len(Y_train['music_category'].unique()))
model.summary()

Model: "Genres"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 11)                253       
                                                                 
Total params: 253 (1012.00 Byte)
Trainable params: 253 (1012.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [102]:
# Fit the model.
history = model.fit(
  x = X_train,   # training examples
  y = Y_train,   #labels
  epochs=5,             # number of passes through the training data
  batch_size=64,        # mini-batch size for SGD
  validation_split=0.1, # use a fraction of the examples for validation -- DO WE STILL NEED THIS IF VALIDATION SPLIT
  verbose=1             # display some progress output during training
  )

# Convert the return value into a DataFrame so we can see the train loss 
# and binary accuracy after every epoch.
history = pd.DataFrame(history.history)
display(history)

Epoch 1/5
400/400 [==============================] - 1s 985us/step - loss: 54.4848 - accuracy: 0.2312 - val_loss: 62.8444 - val_accuracy: 0.1795
Epoch 2/5
400/400 [==============================] - 0s 787us/step - loss: 50.2543 - accuracy: 0.2376 - val_loss: 37.7028 - val_accuracy: 0.1823
Epoch 3/5
400/400 [==============================] - 0s 789us/step - loss: 48.8607 - accuracy: 0.2388 - val_loss: 44.0321 - val_accuracy: 0.4096
Epoch 4/5
400/400 [==============================] - 0s 784us/step - loss: 46.9103 - accuracy: 0.2437 - val_loss: 26.6610 - val_accuracy: 0.3874
Epoch 5/5
400/400 [==============================] - 0s 774us/step - loss: 47.7023 - accuracy: 0.2421 - val_loss: 25.8470 - val_accuracy: 0.2136


,loss,accuracy,val_loss,val_accuracy
0,54.484772,0.231241,62.844364,0.179451
1,50.254295,0.237615,37.702766,0.182266
2,48.860737,0.238788,44.032101,0.409571
3,46.910324,0.243675,26.661022,0.387403
4,47.702297,0.242072,25.847004,0.213582


##### TESTING for EVENLY distributed classes

In [103]:
model = build_model(num_features = X_train_uniform.shape[1], n_classes = len(Y_train['music_category'].unique()))
model.summary()

Model: "Genres"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 11)                253       
                                                                 
Total params: 253 (1012.00 Byte)
Trainable params: 253 (1012.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [104]:
# Fit the model.
history = model.fit(
  x = X_train_uniform,   # training examples
  y = Y_train_uniform,   #labels
  epochs=5,             # number of passes through the training data
  batch_size=64,        # mini-batch size for SGD
  validation_split=0.1, # use a fraction of the examples for validation -- DO WE STILL NEED THIS IF VALIDATION SPLIT
  verbose=1             # display some progress output during training
  )

# Convert the return value into a DataFrame so we can see the train loss 
# and binary accuracy after every epoch.
history = pd.DataFrame(history.history)
display(history)

Epoch 1/5
49/49 [==============================] - 0s 3ms/step - loss: 65.1287 - accuracy: 0.0894 - val_loss: 164.7680 - val_accuracy: 0.0000e+00
Epoch 2/5
49/49 [==============================] - 0s 1ms/step - loss: 64.4278 - accuracy: 0.1046 - val_loss: 166.7361 - val_accuracy: 0.0000e+00
Epoch 3/5
49/49 [==============================] - 0s 1ms/step - loss: 63.3116 - accuracy: 0.1108 - val_loss: 174.1063 - val_accuracy: 0.0000e+00
Epoch 4/5
49/49 [==============================] - 0s 1ms/step - loss: 61.2094 - accuracy: 0.1082 - val_loss: 165.4035 - val_accuracy: 0.0000e+00
Epoch 5/5
49/49 [==============================] - 0s 1ms/step - loss: 61.9481 - accuracy: 0.1114 - val_loss: 154.2659 - val_accuracy: 0.0000e+00


,loss,accuracy,val_loss,val_accuracy
0,65.128716,0.089378,164.767960,0.0
1,64.427788,0.104598,166.736115,0.0
2,63.311558,0.110751,174.106293,0.0
3,61.209377,0.108161,165.403503,0.0
4,61.948132,0.111399,154.265884,0.0


### Feature Scaling & Normalization

In [107]:
X_train.describe()

,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,D,D#,E,F,F#,G,G#,A,A#,B
count,28417.000000,28417.000000,28417.000000,28417.000000,28417.000000,28417.000000,28417.000000,28417.000000,28417.000000,28417.000000,...,28417.000000,28417.000000,28417.000000,28417.000000,28417.00000,28417.000000,28417.000000,28417.000000,28417.000000,28417.000000
mean,0.563809,0.656935,-8.183055,0.633952,0.100604,0.297483,0.189521,0.210896,0.458677,123.132188,...,0.101066,0.029243,0.078650,0.080269,0.07221,0.115424,0.063554,0.098321,0.067108,0.078720
std,0.179260,0.255036,4.942298,0.481731,0.137034,0.330553,0.332660,0.184895,0.259785,29.673745,...,0.301422,0.168490,0.269197,0.271714,0.25884,0.319538,0.243960,0.297754,0.250213,0.269307
min,0.000000,0.000020,-40.558000,0.000000,0.000000,0.000000,0.000000,0.011600,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.448000,0.481000,-10.104000,0.000000,0.036500,0.008340,0.000000,0.097400,0.242000,100.013000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.574000,0.702000,-6.950000,1.000000,0.052100,0.140000,0.000126,0.131000,0.443000,122.392000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.696000,0.877000,-4.899000,1.000000,0.099300,0.569000,0.192000,0.275000,0.666000,141.868000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.984000,1.000000,4.532000,1.000000,0.963000,0.996000,0.999000,0.994000,0.995000,220.081000,...,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


In [108]:
# YOUR CODE HERE
X_train_features_norm = (X_train - X_train.mean())/X_train.std() 
X_test_features_norm = (X_test - X_train.mean())/X_train.std()
#only train features shown since we're not supposed to look at test
X_train_features_norm.describe()

,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,D,D#,E,F,F#,G,G#,A,A#,B
count,2.841700e+04,2.841700e+04,2.841700e+04,2.841700e+04,2.841700e+04,2.841700e+04,2.841700e+04,2.841700e+04,2.841700e+04,2.841700e+04,...,2.841700e+04,2.841700e+04,2.841700e+04,2.841700e+04,2.841700e+04,2.841700e+04,2.841700e+04,2.841700e+04,2.841700e+04,2.841700e+04
mean,-4.870807e-16,-4.150688e-17,-4.050671e-17,-1.187697e-17,-5.000829e-18,-2.142855e-16,-8.351384e-17,5.300878e-17,-1.620268e-16,1.250207e-16,...,-4.863306e-17,-2.312883e-17,-4.650771e-17,-1.500249e-17,-3.675609e-17,4.375725e-17,4.100679e-17,-4.850804e-17,-5.375891e-18,-4.975824e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.145199e+00,-2.575772e+00,-6.550585e+00,-1.315985e+00,-7.341533e-01,-8.999554e-01,-5.697121e-01,-1.077886e+00,-1.765603e+00,-4.149533e+00,...,-3.352985e-01,-1.735595e-01,-2.921660e-01,-2.954170e-01,-2.789762e-01,-3.612206e-01,-2.605077e-01,-3.302103e-01,-2.682025e-01,-2.923079e-01
25%,-6.460406e-01,-6.898438e-01,-3.886744e-01,-1.315985e+00,-4.677962e-01,-8.747250e-01,-5.697121e-01,-6.138398e-01,-8.340630e-01,-7.791126e-01,...,-3.352985e-01,-1.735595e-01,-2.921660e-01,-2.954170e-01,-2.789762e-01,-3.612206e-01,-2.605077e-01,-3.302103e-01,-2.682025e-01,-2.923079e-01
50%,5.684784e-02,1.766999e-01,2.494903e-01,7.598601e-01,-3.539560e-01,-4.764233e-01,-5.693333e-01,-4.321153e-01,-6.034582e-02,-2.494420e-02,...,-3.352985e-01,-1.735595e-01,-2.921660e-01,-2.954170e-01,-2.789762e-01,-3.612206e-01,-2.605077e-01,-3.302103e-01,-2.682025e-01,-2.923079e-01
75%,7.374224e-01,8.628771e-01,6.644794e-01,7.598601e-01,-9.516095e-03,8.214003e-01,7.453161e-03,3.467043e-01,7.980568e-01,6.313936e-01,...,-3.352985e-01,-1.735595e-01,-2.921660e-01,-2.954170e-01,-2.789762e-01,-3.612206e-01,-2.605077e-01,-3.302103e-01,-2.682025e-01,-2.923079e-01
max,2.344025e+00,1.345162e+00,2.572701e+00,7.598601e-01,6.293295e+00,2.113173e+00,2.433351e+00,4.235394e+00,2.064489e+00,3.267158e+00,...,2.982312e+00,5.761509e+00,3.422591e+00,3.384927e+00,3.584409e+00,2.768294e+00,3.838522e+00,3.028267e+00,3.728395e+00,3.420930e+00


### Model 2

In [109]:
def build_model2(num_features, n_classes,learning_rate=0.01):
    """Build a TF logistic regression model using Keras.
    
    Args:
    learning_rate: The desired learning rate for SGD.
    
    Returns:
    model: A tf.keras model (graph).
    """
    # Random Seed + Clear Session
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)
    
    # Build a model using keras.Sequential.
    model = keras.Sequential(name = 'Genres')
    
    # Keras layers processing
    model.add(tf.keras.layers.InputLayer(input_shape=num_features))

    #adding extra layer
    model.add(keras.layers.Dense(
            units=256,
            activation = 'relu'))

    #add extra layer
    model.add(keras.layers.Dense(
            units=128,
            activation = 'relu'))
    
    # This layer constructs the linear set of parameters for each input feature
    # (as well as a bias), and applies a sigmoid to the result. The result is
    # binary logistic regression.
    model.add(keras.layers.Dense(
      units=n_classes,
      activation='softmax'))

    # Use the SGD optimizer as usual.
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    
    # We specify the binary_crossentropy loss (equivalent to log loss).
    # Notice that we are including 'binary accuracy' as one of the metrics that we
    # ask Tensorflow to report when evaluating the model.
    model.compile(loss='sparse_categorical_crossentropy', 
                optimizer=optimizer, 
                metrics=['accuracy'])
    
    return model

In [110]:
model2 = build_model2(num_features = X_train_features_norm.shape[1], n_classes = len(Y_train['music_category'].unique()))
model2.summary()

Model: "Genres"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               5888      
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 11)                1419      
                                                                 
Total params: 40203 (157.04 KB)
Trainable params: 40203 (157.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [111]:
# Fit the model.
history2 = model2.fit(
  x = X_train_features_norm,   # training examples
  y = Y_train,   #labels
  epochs=10,             # number of passes through the training data
  batch_size=128,        # mini-batch size for SGD
  validation_split=0.1, # use a fraction of the examples for validation -- DO WE STILL NEED THIS IF VALIDATION SPLIT
  verbose=1             # display some progress output during training
  )

# Convert the return value into a DataFrame so we can see the train loss 
# and binary accuracy after every epoch.
history2 = pd.DataFrame(history2.history)
display(history)

Epoch 1/10
200/200 [==============================] - 1s 2ms/step - loss: 1.8976 - accuracy: 0.3935 - val_loss: 1.7752 - val_accuracy: 0.4187
Epoch 2/10
200/200 [==============================] - 0s 1ms/step - loss: 1.7040 - accuracy: 0.4344 - val_loss: 1.6852 - val_accuracy: 0.4426
Epoch 3/10
200/200 [==============================] - 0s 1ms/step - loss: 1.6284 - accuracy: 0.4583 - val_loss: 1.6278 - val_accuracy: 0.4553
Epoch 4/10
200/200 [==============================] - 0s 1ms/step - loss: 1.5773 - accuracy: 0.4693 - val_loss: 1.5875 - val_accuracy: 0.4659
Epoch 5/10
200/200 [==============================] - 0s 1ms/step - loss: 1.5408 - accuracy: 0.4749 - val_loss: 1.5583 - val_accuracy: 0.4729
Epoch 6/10
200/200 [==============================] - 0s 1ms/step - loss: 1.5137 - accuracy: 0.4767 - val_loss: 1.5371 - val_accuracy: 0.4736
Epoch 7/10
200/200 [==============================] - 0s 1ms/step - loss: 1.4928 - accuracy: 0.4777 - val_loss: 1.5198 - val_accuracy: 0.4733
Epoch 

,loss,accuracy,val_loss,val_accuracy
0,65.128716,0.089378,164.767960,0.0
1,64.427788,0.104598,166.736115,0.0
2,63.311558,0.110751,174.106293,0.0
3,61.209377,0.108161,165.403503,0.0
4,61.948132,0.111399,154.265884,0.0


### Testing - Feature Selection

In [112]:
X_train_features_norm.columns

Index(['danceability', 'energy', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'C',
       'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'],
      dtype='object')

In [113]:
features = ['danceability','energy', 'loudness', 'valence', 'tempo']
X_train_features_norm = X_train_features_norm[features]

In [114]:
X_train_features_norm.head()

,danceability,energy,loudness,valence,tempo
0,1.072131,-0.544766,0.254751,-0.726281,-0.542472
1,-0.958435,0.243357,0.501195,1.367759,0.576463
2,0.246516,0.541354,0.649304,1.733446,-0.112395
3,1.819647,0.133569,-0.211227,0.925085,-0.779517
4,1.088867,-0.685923,-0.189374,1.221484,-1.453682


In [115]:
model2 = build_model2(num_features = len(features), n_classes = len(Y_train['music_category'].unique()))
model2.summary()

Model: "Genres"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               1536      
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 11)                1419      
                                                                 
Total params: 35851 (140.04 KB)
Trainable params: 35851 (140.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [116]:
# Fit the model.
history2 = model2.fit(
  x = X_train_features_norm,   # training examples
  y = Y_train,   #labels
  epochs=10,             # number of passes through the training data
  batch_size=128,        # mini-batch size for SGD
  validation_split=0.1, # use a fraction of the examples for validation -- DO WE STILL NEED THIS IF VALIDATION SPLIT
  verbose=1             # display some progress output during training
  )

# Convert the return value into a DataFrame so we can see the train loss 
# and binary accuracy after every epoch.
history2 = pd.DataFrame(history2.history)
display(history)

Epoch 1/10
200/200 [==============================] - 1s 2ms/step - loss: 1.9907 - accuracy: 0.3958 - val_loss: 1.8240 - val_accuracy: 0.4099
Epoch 2/10
200/200 [==============================] - 0s 1ms/step - loss: 1.7624 - accuracy: 0.4210 - val_loss: 1.7307 - val_accuracy: 0.4282
Epoch 3/10
200/200 [==============================] - 0s 1ms/step - loss: 1.6924 - accuracy: 0.4453 - val_loss: 1.6803 - val_accuracy: 0.4433
Epoch 4/10
200/200 [==============================] - 0s 1ms/step - loss: 1.6500 - accuracy: 0.4540 - val_loss: 1.6471 - val_accuracy: 0.4493
Epoch 5/10
200/200 [==============================] - 0s 1ms/step - loss: 1.6212 - accuracy: 0.4565 - val_loss: 1.6234 - val_accuracy: 0.4525
Epoch 6/10
200/200 [==============================] - 0s 1ms/step - loss: 1.6002 - accuracy: 0.4580 - val_loss: 1.6061 - val_accuracy: 0.4546
Epoch 7/10
200/200 [==============================] - 0s 1ms/step - loss: 1.5839 - accuracy: 0.4600 - val_loss: 1.5917 - val_accuracy: 0.4560
Epoch 

,loss,accuracy,val_loss,val_accuracy
0,65.128716,0.089378,164.767960,0.0
1,64.427788,0.104598,166.736115,0.0
2,63.311558,0.110751,174.106293,0.0
3,61.209377,0.108161,165.403503,0.0
4,61.948132,0.111399,154.265884,0.0


In [117]:
X_test_features_norm = X_test_features_norm[features]
model2.predict(X_test_features_norm)

296/296 [==============================] - 0s 652us/step


array([[3.5477951e-01, 9.9735469e-02, 1.9490340e-01, ..., 2.0145010e-02,
        4.3456461e-02, 1.5037460e-02],
       [2.3781808e-01, 6.9914209e-03, 3.0287290e-01, ..., 2.9761717e-03,
        8.5304659e-03, 1.9787184e-03],
       [3.1469461e-01, 4.3431979e-02, 3.7975064e-01, ..., 1.2555168e-02,
        4.3780994e-02, 2.6898198e-02],
       ...,
       [4.1330349e-01, 1.2707220e-01, 9.6689254e-02, ..., 2.3585441e-02,
        2.9136784e-02, 9.5483242e-03],
       [4.4454592e-01, 2.0527979e-02, 9.7807519e-02, ..., 1.3152970e-02,
        1.3619703e-02, 2.0951215e-02],
       [7.2339255e-01, 1.7907671e-03, 1.2162405e-01, ..., 5.4840656e-04,
        1.3400431e-03, 8.7768294e-02]], dtype=float32)

In [118]:
print("Evaluate on test data")
results = model2.evaluate(X_test_features_norm, Y_test, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
74/74 [==============================] - 0s 781us/step - loss: 1.5495 - accuracy: 0.4647
test loss, test acc: [1.5494897365570068, 0.4647381901741028]
